In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist #importation des données + normalisation
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
modele=tf.keras.Sequential()
modele.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(28, 28, 1))) #32=dim espace de sortie, strides, prend les 28x28
modele.add(tf.keras.layers.MaxPooling2D((2, 2))) #reduction de la dimension spatiale en sortie de la couche de convolution, mise en commun
modele.add(tf.keras.layers.Flatten()) #écrasement en vecteur
modele.add(tf.keras.layers.Dense(100, activation='relu'))
modele.add(tf.keras.layers.Dense(10, activation='softmax')) #puis activation à 10 neurones puisque chiffres 1 à 10
opt = tf.keras.optimizers.SGD(lr=0.1)
loss = tf.keras.losses.CategoricalCrossentropy()

In [6]:
modele.compile(opt, loss)

In [7]:
modele.fit(x_train, y_train, epochs=2)

ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (60000, 28, 28)